In [2]:
import numpy as np
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import binom

#### HELPER FUNCTIONS
def generate_random_data(Nsample,NDP=128):
    X = np.random.uniform(0,1.0,(NDP,Nsample))
    W = np.random.uniform(-1.0,1.0,(NDP,Nsample))
    muX = np.mean(X)
    muW = np.mean(W)
    varX = np.var(X)
    varW = np.var(W)
    return [X,W,muX,muW,varX,varW]

def generate_normal_data(Nsample,NDP,scale):
    X1 = np.random.normal(0,scale,(NDP,Nsample))
    X2 = np.random.normal(0,scale,(NDP,Nsample))
    W1 = np.random.normal(0,scale,(NDP,Nsample))
    W2 = np.random.normal(0,scale,(NDP,Nsample))
    X1 = np.clip(X1,0,1)
    X2 = np.clip(X2,0,1)
    W1 = np.clip(W1,-1,1)
    W2 = np.clip(W2,-1,1)
    return [X1,X2,W1,W2]


def quantizeWeight(W,BW):
    W = np.minimum(W,1.0-np.power(2.0,-(BW-1.0)))
    Wbs = []
    Wbi = np.less(W,0).astype(float)
    Wbs.append(Wbi)
    W = (W + Wbi)
    for i in range(BW-1):
        Wbi = np.greater_equal(W,0.5).astype(float)
        Wbs.append(Wbi)
        W = 2.0*W - Wbi
    carry = np.greater_equal(W,0.5).astype(float)
    for i in range(BW):#-1):
        j=BW-1-i
        Wbs[j] = Wbs[j]+carry
        carry = np.greater(Wbs[j],1.5).astype(float)
        Wbs[j] = Wbs[j]*np.not_equal(Wbs[j],2.0)
    return Wbs

def quantizeInput(X,BX):
    X = np.minimum(X,1.0-np.power(2.0,-BX))
    Xbs = []
    for i in range(BX):
        Xbi = np.greater_equal(X,0.5).astype(float)
        Xbs.append(Xbi)
        X = 2.0*X - Xbi
    carry = np.greater_equal(X,0.5).astype(float)
    for i in range(BX):
        j=BX-1-i
        Xbs[j] = Xbs[j]+carry
        carry = np.greater(Xbs[j],1.5).astype(float)
        Xbs[j] = Xbs[j]*np.not_equal(Xbs[j],2.0)
    return Xbs

def reconstructWeight(Wbs,BW):
    W=np.zeros_like(Wbs[0])
    for j in range(BW):
        multiplier = np.power(0.5,j)
        if (j==0):
            multiplier=-1.0
        W+=Wbs[j]*multiplier
    return W

def reconstructInput(Xbs,BX):
    X=np.zeros_like(Xbs[0])
    for l in range(BX):
        multiplier = np.power(0.5,l+1.0)
        X+=Xbs[l]*multiplier
    return X

def SNRformula(Yt,Yq):
    return np.maximum(0,10.0*np.log10(np.true_divide(np.sum(np.square(Yt)),np.sum(np.square(Yt-Yq)))))

##### END HELPER FUNCTIONS

### SNR functions
def dotProductSingular(X,W,BX,BW,kclip,sigma_D):
    Xq = reconstructInput(quantizeInput(X,BX),BX)
    Wbs = quantizeWeight(W,BW)
    for b in range(BW-1):
        Wbs[b+1] = Wbs[b+1]*(1+np.random.normal(0,sigma_D,Wbs[b+1].shape))
    Wq = reconstructWeight(Wbs,BW)
    Wmax = kclip*np.power(2.0,-(BW-1))
    #print(Wmax)
    Wq = np.clip(Wq,-Wmax,Wmax)
    Yq = np.sum(Xq*Wq,axis=0)
    return Yq

def dotProductTotal(X,W,BX,BWs,NBWs,NVWLs,kclip,sigma_D):
    Yt = np.sum(X*W,axis=0)
    y_vs_VWL = []
    for i in range(NVWLs):
        y_vs_BW = []
        for j in range(NBWs):
            Yq = dotProductSingular(X,W,BX,int(BWs[j]),kclip[i],sigma_D[i])
            y_vs_BW.append(Yq)
        y_vs_VWL.append(y_vs_BW)
    return Yt,y_vs_VWL

def computeSNR1(Yt, y_vs_VWL, NVWLs, NBWs):
    SNR1_vs_VWL = []
    for i in range(NVWLs):
        SNR1_vs_BW = []
        for j in range(NBWs):
            current_SNR = SNRformula(Yt,y_vs_VWL[i][j])
            SNR1_vs_BW.append(current_SNR)
        SNR1_vs_VWL.append(SNR1_vs_BW)
    return SNR1_vs_VWL

def getBWOPTandSNROPT(SNR1s,BWs,NVWLs):
    BWOPT=[]
    SNROPT = []
    for i in range(NVWLs):
        current_SNR = np.asarray(SNR1s[i])
        current_SNROPT = np.amax(current_SNR)
        current_BWINDEX = np.argmax(current_SNR)
        SNROPT.append(current_SNROPT)
        BWOPT.append(BWs[current_BWINDEX])
    return BWOPT,SNROPT

def predictSNR1(X,W,muX,muW,varX,varW,BX,BWs,NBWs,NVWLs,kclip,sigma_D):
    xmrs2 = np.square(muX) + varX
    deltaW = np.power(2.0,-(BWs-1))
    deltaX = np.power(2.0,-BX)
    SNR1_vs_VWL = []
    for i in range(NVWLs):
        SNR1_vs_BW = []
        for j in range(NBWs):
            sig2_qw = np.square(deltaW[j])/varW
            sig2_qx = np.square(deltaX)/xmrs2
            sig2_clip = np.power(kclip[i],-2.0)*np.power(4.0,BWs[j])*np.square(np.maximum(0,1-2*kclip[i]*np.power(2.0,-BWs[j])))
            sig2_elec = 2*(1-np.power(4.0,-(BWs[j]-1)))*np.square(sigma_D[i])/varW
            current_SNR = np.maximum(0,10.0*np.log10(12/(sig2_qw+sig2_qx+sig2_clip+sig2_elec)))
            SNR1_vs_BW.append(current_SNR)
        SNR1_vs_VWL.append(SNR1_vs_BW)
    return SNR1_vs_VWL

### Key variables Definitions
NDP = 128
Nsample = 1000
#VWLs = np.asarray([0.55,0.6,0.65,0.7,0.75,0.8])
VWLs = np.arange(0.6,0.9,0.01)
NVWLs = VWLs.size
BX = 6
#BWs = np.asarray([4,5,6,7,8])
BWs = np.arange(4,12,1)
NBWs = BWs.size
VBLMAX = 0.8
T0 = 100e-12
kn = 220e-6
Vt = 0.4
alpha = 1.8
CBL = 270e-15
Icell = kn*np.power(VWLs-Vt,alpha)
delta_VBL_LSB = T0*Icell/CBL
kclip = VBLMAX/delta_VBL_LSB
#kclip = 10000+VBLMAX/delta_VBL_LSB
sigma_Vt = 23.8e-3
sigma_D = alpha*sigma_Vt/(VWLs-Vt)
#sigma_D = sigma_D-sigma_D

### Data Generation and operations
X,W,muX,muW,varX,varW = generate_random_data(Nsample,NDP)
Yt, y_vs_VWL = dotProductTotal(X,W,BX,BWs,NBWs,NVWLs,kclip,sigma_D)
SNR1s_computed = np.asarray(computeSNR1(Yt, y_vs_VWL, NVWLs, NBWs))
SNR1s_predicted = np.asarray(predictSNR1(X,W,muX,muW,varX,varW,BX,BWs,NBWs,NVWLs,kclip,sigma_D))
#BWOPT, SNROPT = getBWOPTandSNROPT(SNR1s_computed,BWs,NVWLs)


### PLOTTING
fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.set_xlabel(r'$V_\mathrm{WL}$ (V)',fontsize=20)
ax1.set_ylabel(r'SNR$_\mathrm{A(dB)}$ ',fontsize=20)

lines = []
for i in range(NBWs):
    if BWs[i] in [6,7,8]:#,8]:
        line, = ax1.plot(VWLs,SNR1s_predicted[:,i],linewidth=2,label=r'E: $B_w$='+str(BWs[i]),marker='o')
        lines.append(line)
        color = line.get_color()
        line, = ax1.plot(VWLs,SNR1s_computed[:,i],linewidth=2,label=r'S: $B_w$='+str(BWs[i]),linestyle='--',color=color,marker='s')
        lines.append(line)
#ax1.tick_params(axis='y', labelcolor=color)
ax1.legend(handles=lines,loc=0, fontsize=15)#bbox_to_anchor=(1,1),
ax1.grid()
ax1.tick_params(axis='both',labelsize=15)
#ax1.set_xticks([5,6,7])
plt.savefig('SNRvsVWL.pdf',bbox_inches='tight')
plt.clf()

fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.set_xlabel(r'$B_w$',fontsize=20)
ax1.set_ylabel(r'SNR$_\mathrm{A(dB)}$ ',fontsize=20)
lines = []
line, = ax1.plot(BWs,SNR1s_predicted[0,:],linewidth=2,label=r'E: $V_{WL}$=%.1f V'%(VWLs[0]),marker='o')
lines.append(line)
color = line.get_color()
line, = ax1.plot(BWs,SNR1s_computed[0,:],linewidth=2,label=r'S: $V_{WL}$=%.1f V'%(VWLs[0]),linestyle='--',color=color,marker='s')
lines.append(line)
line, = ax1.plot(BWs,SNR1s_predicted[10,:],linewidth=2,label=r'E: $V_{WL}$=%.1f V'%(VWLs[10]),marker='o')
lines.append(line)
color = line.get_color()
line, = ax1.plot(BWs,SNR1s_computed[10,:],linewidth=2,label=r'S: $V_{WL}$=%.1f V'%(VWLs[10]),linestyle='--',color=color,marker='s')
lines.append(line)
line, = ax1.plot(BWs,SNR1s_predicted[-1,:],linewidth=2,label=r'E: $V_{WL}$=%.1f V'%(VWLs[-1]),marker='o')
lines.append(line)
color = line.get_color()
line, = ax1.plot(BWs,SNR1s_computed[-1,:],linewidth=2,label=r'S: $V_{WL}$=%.1f V'%(VWLs[-1]),linestyle='--',color=color,marker='s')
lines.append(line)
#ax1.tick_params(axis='y', labelcolor=color)
ax1.legend(handles=lines,loc=0, fontsize=15)
ax1.grid()
ax1.tick_params(axis='both',labelsize=15)

ax1.set_ylim(bottom=0)
ax1.set_ylim(top=25)

plt.savefig('SNRvsBW.pdf',bbox_inches='tight')
plt.clf()
### PREVIOUS CODE
'''
def predict_sqnr(BW,BX,VWL):
    sig2_qx = np.power(2.0,-BX)*3.0
    sig2_qw = np.power(2.0,1.0-BW)*3.0
    kclip = 6.41*np.power(VWL-0.4,-1.8)
    sig2_clip = np.power(kclip,-2.0)*np.power(2.0,2.0*BW)*np.square(np.maximum(0.0,1-2.0*kclip*np.power(2.0,-BW)))
    sig2_fr = 12.0*np.square(np.true_divide(0.0212,VWL-0.4))
    return np.maximum(0.0,10.0*np.log10(np.true_divide(12.0,sig2_qx+sig2_qw+sig2_clip+sig2_fr)))

BX=6.0
fig,ax=plt.subplots()
lines=[]
for VWL in np.arange(0.5,0.81,0.05).tolist():
    SNRs = []
    BWs = []
    for BW in range(1,20):
        BWs.append(BW)
        SNRs.append(predict_sqnr(BW,BX,VWL))
    line, = ax.plot(BWs,SNRs,label=r'$V_{WL} = $'+str(VWL),linewidth=2)
    lines.append(line)
plt.legend(handles=lines,loc=0,fontsize=15)
ax.grid()
ax.set_xlabel(r'$B_W$ (bits)',fontsize=20)
ax.set_ylabel('SNR (dB)',fontsize=20)
ax.yaxis.label.set_fontsize(15)
ax.xaxis.label.set_fontsize(15)
ax.set_xticks(np.arange(1,20))
plt.savefig('fr_noise_impact_BX_'+str(BX)+'.png',bbox_inches='tight')
plt.clf()
'''


"\ndef predict_sqnr(BW,BX,VWL):\n    sig2_qx = np.power(2.0,-BX)*3.0\n    sig2_qw = np.power(2.0,1.0-BW)*3.0\n    kclip = 6.41*np.power(VWL-0.4,-1.8)\n    sig2_clip = np.power(kclip,-2.0)*np.power(2.0,2.0*BW)*np.square(np.maximum(0.0,1-2.0*kclip*np.power(2.0,-BW)))\n    sig2_fr = 12.0*np.square(np.true_divide(0.0212,VWL-0.4))\n    return np.maximum(0.0,10.0*np.log10(np.true_divide(12.0,sig2_qx+sig2_qw+sig2_clip+sig2_fr)))\n\nBX=6.0\nfig,ax=plt.subplots()\nlines=[]\nfor VWL in np.arange(0.5,0.81,0.05).tolist():\n    SNRs = []\n    BWs = []\n    for BW in range(1,20):\n        BWs.append(BW)\n        SNRs.append(predict_sqnr(BW,BX,VWL))\n    line, = ax.plot(BWs,SNRs,label=r'$V_{WL} = $'+str(VWL),linewidth=2)\n    lines.append(line)\nplt.legend(handles=lines,loc=0,fontsize=15)\nax.grid()\nax.set_xlabel(r'$B_W$ (bits)',fontsize=20)\nax.set_ylabel('SNR (dB)',fontsize=20)\nax.yaxis.label.set_fontsize(15)\nax.xaxis.label.set_fontsize(15)\nax.set_xticks(np.arange(1,20))\nplt.savefig('fr_noise_

[0.0044969  0.00490969 0.00533851 0.00578321 0.00624366 0.00671972
 0.00721126 0.00771816 0.00824031 0.00877759 0.0093299  0.00989715
 0.01047922 0.01107603 0.01168748 0.0123135  0.01295399 0.01360887
 0.01427807 0.0149615  0.0156591 ]
